# ANÁLISIS DISCRIMINANTE LINEAL

## Realizado por: Pablo Sánchez Cabrera

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import matplotlib.pyplot as plt
import seaborn as sns

Se cargan los datos de `OJ`. Estos datos están ya disponibles en la librería **statsmodels**.

In [3]:
data = sm.datasets.get_rdataset("OJ", "ISLR", cache=True).data

Seleccionamos las variables con las que se ilustra la técnica y se hace un resumen de las variables

In [4]:
var = ['Purchase', 'PriceDiff', 'PctDiscMM', 'PctDiscCH', 'LoyalCH']
data = data.loc[:, var]

data.describe()  #resumen de las variables cuantitativas

,PriceDiff,PctDiscMM,PctDiscCH,LoyalCH
count,1070.000000,1070.000000,1070.000000,1070.000000
mean,0.146486,0.059298,0.027314,0.565782
std,0.271563,0.101760,0.062232,0.307843
min,-0.670000,0.000000,0.000000,0.000011
25%,0.000000,0.000000,0.000000,0.325257
50%,0.230000,0.000000,0.000000,0.600000
75%,0.320000,0.112676,0.000000,0.850873
max,0.640000,0.402010,0.252688,0.999947


Se define el modelo identificando tanto el target como las variables explicativas. 

In [6]:
#target
purchase = data['Purchase']
purchase_cat = ['CH', 'MM'] # categorías del target

# variables con poder discriminante
var_disc = data.drop('Purchase', axis=1)

#Definición y ajuste del modelo
lda=LDA(solver='svd')  # "svd" (descomposición en valores singulares)
modelo_lda=lda.fit(var_disc, purchase) # ajuste 


En este caso se emplea el método `"svd" - descomposición en valores singulares` pero podría emplearse otros como
`"lsqr" - mínimos cuadrados` o `"eigen" - descomposión en valores propios`.

`Probabilidad a priori de los grupos`

In [7]:
pd.DataFrame(modelo_lda.priors_, columns=['Grupos: Prob. a priori'], index=purchase_cat)

,Grupos: Prob. a priori
CH,0.61028
MM,0.38972


`Predicción - Probabilidades a posteriori`

In [9]:
prob = pd.DataFrame(modelo_lda.predict_proba(var_disc), columns=purchase_cat)
clase = pd.DataFrame(modelo_lda.predict(var_disc), columns=['Pred'])
prob_clase = pd.concat([clase, prob], axis=1)   

prob_clase.head(5) #se muestra los primeros 5 registros

,Pred,CH,MM
0,CH,0.636148,0.363852
1,CH,0.645331,0.354669
2,CH,0.920324,0.079676
3,MM,0.233605,0.766395
4,CH,0.952343,0.047657


Como puede verse en la tabla, se elige como clase predicha a la categoría cuya probabilidad sea superior a 0.5.

`Bondad de ajuste`

Se presenta tanto la matriz de confusión como el porcentaje de bien clasificados

In [10]:
pd.crosstab(prob_clase['Pred'], purchase) #matriz de confusión

Purchase,CH,MM
Pred,,
CH,567,101
MM,86,316


In [12]:
round(np.mean(prob_clase['Pred'] == purchase),5)

0.82523